
# MLflow Trace Notebook

This is a notebook for testing MLflow Trace APIs. Currently this implements a set of Python APIs for creating trace/spans, and show the basic UI of rendeing traces in notebook cells. The primary purpose of this notebook is to gather early feedbacks for the APIs and the UI and find any broken pieces before the official private preview.


### API Documentation

#### === Overview ===
In this prototype, we have implemented the following set of Python APIs:
* **Fluent APIs** - for instrumenting Python code without managing parent-child relationship of spans i.e. MLflow will determine the appropriate parent span based on the block scope. The general recommendation is to use fluent API as much as possbile if you can.
* **MLflow Client APIs** - MlflowClient equips similar but more imperative APIs for more advanced use cases like multithreading, callbacks, etc, where you need to manually define the lifecycle of spans and parent-child relationship. For example, if you want to log trace with LangChain callback, you can’t use fluent APIs because component start and end hooks need to be defined as different functions. 

When creating a span with those APIs, it will return a **MLflowSpanWrapper** object, which is an interface to mutate the span object. For example, you can call `set_attributes` method to record additional key-values for the span. 

#### === Fluent APIs ===
There are 3 ways for defining spans using fluent APIs.
1. Decorate a function with `@mlflow.trace`. This is useful when you want to create a span for your own functions.
2. Wrap a function with `mlflow.trace()`. This is useful when you want to create a span for external libraries.
3. Use context manager `with mlflow.start_span()`. This is useful when you want to define more granular spans within a single function.

Please vist the following docstring link for the API spec and arguments.

* [mlflow.trace()](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracing/fluent.py#L17)
* [mlflow.start_span](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracing/fluent.py#L83)


##### Caputuring Inputs and Outputs
If you are using the `@mlflow.trace` decorator or the `mlflow.trace()` wrapper for a function, it will automatically capture the input argument(s) and return value(s) for the function, setting them to `inputs` and `outputs` fields of the span respectively.

When using `with mlflow.start_span()` context manager, you have to manually set `inputs` and `outputs` if you want to record them. This can be done by using `set_inputs()` and `set_outpus()` method of MLflowSpanWrapper object that is returned from the fluent API. Please see **MMLflowSpanWrapper** section below for more details.

#### === MLflow Client APIs ===
MLflowClient exposes a new set of tracing APIs. As mentioned above, those APIs require explicit parent span ID specification to build the parent-child relationship, in contrast to the fluent APIs. Please refer to the following docstring link for the API spec and arguments. Also you can refer to the callback implementation example in the bottom of notebook for the usage.

* [MLflowClient().start_trace()](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracking/client.py#L443)
* [MLflowClient().end_trace()](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracking/client.py#L501)
* [MLflowClient().start_span()](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracking/client.py#L531)
* [MLflowClient().end_span()](https://github.com/mlflow/mlflow/blob/1979b5b9a155c91675b62e759e51a5dbc628203a/mlflow/tracking/client.py#L619)

#### === MLflowSpanWrapper ===
MLflowSpanWrapper provides a set of setter methods to manually mutate the span fields. Some of them are automatically recorded, such as span id, parent span id, start and end time, etc, but sometimes we need manual setting as well.

* `set_inputs()`: Set input variables of the span
* `set_outputs()`: Set output result of the span
* `set_attributes()`: Add additional attributes to the span e.g. token count.
* `set_status()`: Manually set status of the span. Common status change e.g. OK, Error, is handled automatically so you don't have to set it for typical scenario.
* `add_event()`: Record point of time event to the span.

For the usage example, please refer to the **Ex.3** code sample below. The source code for those APIs can be found at [MLflow repository](https://github.com/mlflow/mlflow/blob/tracing/mlflow/tracing/types/wrapper.py#L12).

#### === Examples ===

**Ex.1**

Here is an example of decorating a function with the `@mlflow.trace` decorator.

```

  @mlflow.trace(span_type="CustomModel", attributes={"model_name": "gpt-3.5-turbo"})
  def predict(data, params=None):
      return sum(data)

  predict([1, 2], params={"p": 0.1})


```

Then the generated span contains metadata like this:

```

  {
      "name": "predict"  # Automatically set to function name
      "span_type": "CustomModel",
      "attributes": {
          "model_name": "gpt-3.5-turbo",
      },
      # Input and output data will be logged automatically.
      "inputs": {
          "data": [1, 2],
          "params": {"p": 0.1},
      },
      "outputs": {
          "output": 3
      },
  }


```

**Ex.2**

Here is an example of wrapping a external library's function with the `mlflow.trace()`.

```

  from some_llm_library import client

  mlflow.trace(client.make_request, attributes={"model_name": "gpt-100-ultra"})(query="my-query", temperature=0.1)


```
Then the generated span contains metadata like this:
```

{
    "name": "make_request"  # Automatically set to function name
    "span_type": "CustomModel",
    "attributes": {
        "model_name": "gpt-100-ultra",
    },
    # Input and output data will be logged automatically.
    "inputs": {
        "query": "my-query"
        "params": {"temperature": 0.1},
    },
    "outputs": {
        "output": "Hey, I am Ultra GPT.
    },
}


```


**Ex.3**

Here is an example of using the context manager `with mlflow.start_span()`

```

  x = 1
  y = 2
  with mlflow.start_span(name="sum") as span:
      result = x + y
      # You need to manually set inputs/outputs for this method
      span.set_inputs({"x": x, "y": y})
      span.set_outputs("sum": reuslt)
      span.set_attributes({"key": "value"})

  return result


```
Then the generated span contains metadata like this:
```

{
    "name": "sum",
    "span_type": "unknown",
    "attributes": {
        "key": "value",
    },
    "inputs": {
        "x": 1,
        "y": 2,
    },
    "outputs": {
        "sum": "result"
    },
}


```


# Tracing Custom Pyfunc Model with Fluent APIs

### Defining a Pyfunc Model

In [2]:
import mlflow
from mlflow.deployments import get_deploy_client


class QAChain(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.client = get_deploy_client("databricks")

    @mlflow.trace(name="quickstart-chain")
    def predict(self, model_input, system_prompt, params):
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": model_input[0]["query"]},
        ]

        traced_predict = mlflow.trace(self.client.predict)
        output = traced_predict(
            endpoint=params["model_name"],
            inputs={
                "temperature": params["temperature"],
                "max_tokens": params["max_tokens"],
                "messages": messages,
            },
        )

        with mlflow.start_span(name="_final_answer") as span:
            span.set_inputs({"query": model_input[0]["query"]})

            answer = output["choices"][0]["message"]["content"]

            span.set_outputs({"generated_text": answer})
            # Attributes computed at runtime can be set using the set_attributes() method.
            span.set_attributes(
                {
                    "model_name": params["model_name"],
                    "prompt_tokens": output["usage"]["prompt_tokens"],
                    "completion_tokens": output["usage"]["completion_tokens"],
                    "total_tokens": output["usage"]["total_tokens"],
                }
            )
        return answer

/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


### Run Prediction

In [3]:
SYSTEM_PROMPT = """
You are an assistant for Databricks users. You are answering python, coding, SQL, data engineering, spark, data science, DW and platform, API or infrastructure administration question related to Databricks. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.Use the following pieces of context to answer the question at the end:
"""

model = QAChain()

prediction = model.predict(
    [
        {"query": "What is in MLflow 5.0"},
    ],
    SYSTEM_PROMPT,
    {
        # Using Databricks Foundation Model for easier testing, feel free to replace it.
        "model_name": "databricks-dbrx-instruct",
        "temperature": 0.1,
        "max_tokens": 1000,
    },
)

2024/05/23 20:41:00 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Trace(info=TraceInfo(request_id='e1574bc89124454e93caa7d50bb759d6', experiment_id='0', timestamp_ms=1716464460829, execution_time_ms=1531, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '{"model_input": [{"query": "What is in MLflow 5.0"}], "system_prompt": "\\nYou are an assistant for Databricks users. You are answering python, coding, SQL, data engineering, spark, data science, DW and platform, API or infrastructure administratio...', 'mlflow.traceOutputs': '"I\'m sorry for any confusion, but MLflow is an open-source platform for managing machine learning workflows, and it is not versioned in the same way that Databricks Runtime is. The latest version of MLflow as of my knowledge up to 2021 is MLflow 1...'}, tags={'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///U

In [7]:
from mlflow import MlflowClient

client = MlflowClient()

with mlflow.start_span(name="TEST") as trace:
    span = client.start_span(
        name="CHILD",
        request_id=trace.request_id,
        attributes={"TEST": "TEST"},
        inputs={"foo": "bar"},
        parent_id=trace.span_id,
    )
    client.end_span(span_id=span.span_id, request_id=span.request_id)
    trace.set_inputs({"input": "something"})
    trace.set_attributes({"testing": 123})
    trace.set_outputs(654546455345)

2024/05/23 20:42:35 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Trace(info=TraceInfo(request_id='fce34fac5e0141f39ea5af7fcb38c547', experiment_id='0', timestamp_ms=1716464555051, execution_time_ms=0, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '{"input": "something"}', 'mlflow.traceOutputs': '654546455345'}, tags={'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/fce34fac5e0141f39ea5af7fcb38c547/artifacts', 'mlflow.traceName': 'TEST'}), data=TraceData(spans=[Span(name='TEST', request_id='fce34fac5e0141f39ea5af7fcb38c547', span_id='0x7b7af20ec91a911e', parent_id=None), Span(name='CHILD', request_id='fce34fac5e0141f39ea5af7fcb38c547', span_id='0xeb2eed185b387106', parent_id='0x7b7af20ec91a911e')], request='{"input": "something"}', response='654546455345'))

In [ ]:
# This will generate 2 separate traces, as the client API start_trace will generate a different request_id than the fluent API
# (as expected)
trace = client.start_trace(
    name="PARENT", attributes={"TEST": "TEST"}, inputs="I'm inputting something here"
)

with mlflow.start_span(name="test") as span:
    span.set_attribute("test", "test")
    span.set_attribute("test2", "test2")
    span.set_attribute("test3", "test3")
    span.set_inputs({"input": "something"})
    span.set_attributes({"testing": 123})

client.end_trace(request_id=trace.request_id)

Trace(trace_info=TraceInfo(request_id=263723036695390570778755646286391925911, experiment_id='EXPERIMENT', timestamp_ms=1712283329614, execution_time_ms=7, status=<TraceStatus.OK: 'OK'>, request_metadata={'name': 'PARENT', 'inputs': '"I\'m inputting something here"', 'outputs': ''}, tags={}), trace_data=TraceData(spans=[Span(name='PARENT', context=SpanContext(request_id=263723036695390570778755646286391925911, span_id=2517276266223269133), parent_span_id=None, status=SpanStatus(status_code=<TraceStatus.OK: 'OK'>, description=''), start_time=1712283329614504, end_time=1712283329622481, span_type='UNKNOWN', inputs="I'm inputting something here", outputs=None, attributes={'TEST': 'TEST'}, events=[])]))

In [9]:
mlflow.search_traces()

,request_id,trace,timestamp_ms,status,execution_time_ms,request,response,request_metadata,spans,tags
0,fce34fac5e0141f39ea5af7fcb38c547,Trace(info=TraceInfo(request_id='fce34fac5e014...,1716464555045,TraceStatus.OK,6,"{""input"": ""something""}",654546455345,"{'mlflow.traceInputs': '{""input"": ""something""}...","[{'name': 'TEST', 'context': {'span_id': '0x7b...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
1,a03368ff6cf847c881f938277ee53c88,Trace(info=TraceInfo(request_id='a03368ff6cf84...,1716464546748,TraceStatus.ERROR,9,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0xad...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
2,67ccd905c32a43c1af50e8fd781fde2b,Trace(info=TraceInfo(request_id='67ccd905c32a4...,1716464490166,TraceStatus.ERROR,7,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0x9c...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
3,e1574bc89124454e93caa7d50bb759d6,Trace(info=TraceInfo(request_id='e1574bc891244...,1716464460770,TraceStatus.OK,1590,"{""model_input"": [{""query"": ""What is in MLflow ...","""I'm sorry for any confusion, but MLflow is an...","{'mlflow.traceInputs': '{""model_input"": [{""que...","[{'name': 'quickstart-chain', 'context': {'spa...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."


[Trace(info=TraceInfo(request_id='fce34fac5e0141f39ea5af7fcb38c547', experiment_id='0', timestamp_ms=1716464555045, execution_time_ms=6, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '{"input": "something"}', 'mlflow.traceOutputs': '654546455345'}, tags={'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/fce34fac5e0141f39ea5af7fcb38c547/artifacts', 'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.traceSpans': '[{"name": "TEST", "type": "UNKNOWN", "inputs": ["input"]}, {"name": "CHILD", "type": "UNKNOWN", "inputs": ["foo"]}]', 'mlflow.source.type': 'LOCAL', 'mlflow.traceName': 'TEST'}), data=TraceData(spans=[Span(name='TEST', request_id='fce34fac5e0141f39ea5af7fcb38c547', span_id='0x7b7af20ec91a911e', parent_id=None), Span(name='CHILD', request_id='fce34fac5e0141f39ea5af7

### Manually Display Trace

In [11]:
mlflow.search_traces()

,request_id,trace,timestamp_ms,status,execution_time_ms,request,response,request_metadata,spans,tags
0,fce34fac5e0141f39ea5af7fcb38c547,Trace(info=TraceInfo(request_id='fce34fac5e014...,1716464555045,TraceStatus.OK,6,"{""input"": ""something""}",654546455345,"{'mlflow.traceInputs': '{""input"": ""something""}...","[{'name': 'TEST', 'context': {'span_id': '0x7b...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
1,a03368ff6cf847c881f938277ee53c88,Trace(info=TraceInfo(request_id='a03368ff6cf84...,1716464546748,TraceStatus.ERROR,9,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0xad...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
2,67ccd905c32a43c1af50e8fd781fde2b,Trace(info=TraceInfo(request_id='67ccd905c32a4...,1716464490166,TraceStatus.ERROR,7,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0x9c...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
3,e1574bc89124454e93caa7d50bb759d6,Trace(info=TraceInfo(request_id='e1574bc891244...,1716464460770,TraceStatus.OK,1590,"{""model_input"": [{""query"": ""What is in MLflow ...","""I'm sorry for any confusion, but MLflow is an...","{'mlflow.traceInputs': '{""model_input"": [{""que...","[{'name': 'quickstart-chain', 'context': {'spa...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."


[Trace(info=TraceInfo(request_id='fce34fac5e0141f39ea5af7fcb38c547', experiment_id='0', timestamp_ms=1716464555045, execution_time_ms=6, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '{"input": "something"}', 'mlflow.traceOutputs': '654546455345'}, tags={'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/fce34fac5e0141f39ea5af7fcb38c547/artifacts', 'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.traceSpans': '[{"name": "TEST", "type": "UNKNOWN", "inputs": ["input"]}, {"name": "CHILD", "type": "UNKNOWN", "inputs": ["foo"]}]', 'mlflow.source.type': 'LOCAL', 'mlflow.traceName': 'TEST'}), data=TraceData(spans=[Span(name='TEST', request_id='fce34fac5e0141f39ea5af7fcb38c547', span_id='0x7b7af20ec91a911e', parent_id=None), Span(name='CHILD', request_id='fce34fac5e0141f39ea5af7

# Tracing Langchain Automatically

In [12]:
import mlflow

mlflow.langchain.autolog()

### Defining LangChain RAG 

In [15]:
import os

import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatDatabricks
from langchain_community.embeddings import DatabricksEmbeddings

os.mkdir("resources", exits_ok=True)

DOCUMENT_SAVE_PATH = "resources/document.html"
PAGES = [
    "index.html",
    "introduction/index.html",
    "getting-started/index.html",
]

header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
    "Accept-Language": "en-US,en;q=0.8",
    "Connection": "keep-alive",
}

for subpage_url in PAGES:
    document_url = "https://mlflow.org/docs/latest/" + subpage_url
    page = requests.get(document_url, headers=header)
    soup = BeautifulSoup(page.content, "html.parser")
    if soup.find("div", {"class": "rst-content"}):
        text = soup.find("div", {"class": "rst-content"}).text.replace("\n", " ")
        with open(DOCUMENT_SAVE_PATH, "a") as file:
            file.write(text)

dbrx_llm = ChatDatabricks(
    endpoint="databricks-dbrx-instruct",
    extra_params={"temperature": 0},
    allow_dangerous_deserialization=True,
)

FAISS_PERSIST_DIR = "resources/faiss_database"

raw_documents = TextLoader(DOCUMENT_SAVE_PATH).load()
document_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
document_chunks = document_splitter.split_documents(raw_documents)
faiss_database = FAISS.from_documents(
    document_chunks, DatabricksEmbeddings(endpoint="databricks-bge-large-en")
)

# Save the FAISS database to the specified directory
faiss_database.save_local(FAISS_PERSIST_DIR)

# Create QA chain and save
dbrx_qa = RetrievalQA.from_llm(llm=dbrx_llm, retriever=faiss_database.as_retriever(top_k=3))

### Run Prediction

In [16]:
import mlflow

prediction = dbrx_qa.invoke("How can I write a custom pyfunc?")

2024/05/23 20:44:46 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Trace(info=TraceInfo(request_id='6e75bc31c4314139802313db7af48647', experiment_id='0', timestamp_ms=1716464686013, execution_time_ms=6055, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '{"query": "How can I write a custom pyfunc?"}', 'mlflow.traceOutputs': '{"result": "To create a custom pyfunc model in MLflow, you can follow these steps:\\n\\n1. Create a new directory for your custom pyfunc model. This directory should contain the following files:\\n* `MLmodel`: A configuration file that specifies the ...'}, tags={'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/6e75bc31c4314139802313db7af48647/artifacts', 'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.traceName': 'RetrievalQA'}), data=TraceData(spans=[Span(name='RetrievalQA', request_

### Connecting to Custom Parent Span

In [17]:
import mlflow

with mlflow.start_span(name="Custom Root Scope") as root_span:
    query = "What is MLflow?"
    response = dbrx_qa.invoke(query)
    root_span.set_inputs(query)
    root_span.set_outputs(response)

2024/05/23 20:44:52 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Trace(info=TraceInfo(request_id='bb1622fe7f764b669a9a103e614dc04d', experiment_id='0', timestamp_ms=1716464692090, execution_time_ms=1895, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '"What is MLflow?"', 'mlflow.traceOutputs': '{"query": "What is MLflow?", "result": "MLflow is an open-source platform designed to help machine learning (ML) practitioners and teams manage the complexities of the machine learning process. It focuses on the full lifecycle for ML projects, ens...'}, tags={'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/bb1622fe7f764b669a9a103e614dc04d/artifacts', 'mlflow.traceName': 'Custom Root Scope'}), data=TraceData(spans=[Span(name='Custom Root Scope', request_id='bb1622fe7f764b6

In [18]:
mlflow.search_traces()

,request_id,trace,timestamp_ms,status,execution_time_ms,request,response,request_metadata,spans,tags
0,bb1622fe7f764b669a9a103e614dc04d,Trace(info=TraceInfo(request_id='bb1622fe7f764...,1716464692084,TraceStatus.OK,1901,"""What is MLflow?""","{""query"": ""What is MLflow?"", ""result"": ""MLflow...","{'mlflow.traceInputs': '""What is MLflow?""', 'm...","[{'name': 'Custom Root Scope', 'context': {'sp...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
1,6e75bc31c4314139802313db7af48647,Trace(info=TraceInfo(request_id='6e75bc31c4314...,1716464686007,TraceStatus.OK,6061,"{""query"": ""How can I write a custom pyfunc?""}","{""result"": ""To create a custom pyfunc model in...","{'mlflow.traceInputs': '{""query"": ""How can I w...","[{'name': 'RetrievalQA', 'context': {'span_id'...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
2,fce34fac5e0141f39ea5af7fcb38c547,Trace(info=TraceInfo(request_id='fce34fac5e014...,1716464555045,TraceStatus.OK,6,"{""input"": ""something""}",654546455345,"{'mlflow.traceInputs': '{""input"": ""something""}...","[{'name': 'TEST', 'context': {'span_id': '0x7b...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
3,a03368ff6cf847c881f938277ee53c88,Trace(info=TraceInfo(request_id='a03368ff6cf84...,1716464546748,TraceStatus.ERROR,9,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0xad...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
4,67ccd905c32a43c1af50e8fd781fde2b,Trace(info=TraceInfo(request_id='67ccd905c32a4...,1716464490166,TraceStatus.ERROR,7,None,None,"{'mlflow.traceInputs': '', 'mlflow.traceOutput...","[{'name': 'TEST', 'context': {'span_id': '0x9c...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."
5,e1574bc89124454e93caa7d50bb759d6,Trace(info=TraceInfo(request_id='e1574bc891244...,1716464460770,TraceStatus.OK,1590,"{""model_input"": [{""query"": ""What is in MLflow ...","""I'm sorry for any confusion, but MLflow is an...","{'mlflow.traceInputs': '{""model_input"": [{""que...","[{'name': 'quickstart-chain', 'context': {'spa...","{'mlflow.trace_schema.version': '2', 'mlflow.a..."


[Trace(info=TraceInfo(request_id='bb1622fe7f764b669a9a103e614dc04d', experiment_id='0', timestamp_ms=1716464692084, execution_time_ms=1901, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.traceInputs': '"What is MLflow?"', 'mlflow.traceOutputs': '{"query": "What is MLflow?", "result": "MLflow is an open-source platform designed to help machine learning (ML) practitioners and teams manage the complexities of the machine learning process. It focuses on the full lifecycle for ML projects, ens...'}, tags={'mlflow.trace_schema.version': '2', 'mlflow.artifactLocation': 'file:///Users/harutaka.kawamura/Desktop/repositories/mlflow/mlruns/0/traces/bb1622fe7f764b669a9a103e614dc04d/artifacts', 'mlflow.source.name': '/Users/harutaka.kawamura/.pyenv/versions/miniconda3-latest/envs/mlflow/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.traceSpans': '[{"name": "Custom Root Scope", "type": "UNKNOWN", "outputs": ["query", "result"]}, {"name": "RetrievalQA", "type": "CHAIN", "inputs